# Examples

This jupyter notebook contains examples of running the various functions available in _archeryutils_.
Where different options exist for the examples they are listed. Users are encouraged to re-run various cells of this notebook with different options to explore the full functionality of the code.

## 0. Getting set up

To start off using archeryutils we need to import it.
Assuming it has been installed into the local environment according to the repository documentation this can be done as follows:

In [ ]:
import archeryutils as au

## 1. Basic Building Blocks
The basic building blocks of the archeryutils package are the `targets.Target` and the `rounds.Round` class.

### Target

A target is defined with the following attributes:
- Face diameter [m]
- Scoring system
- Distance [default m]

The following scoring systems are possible:
- `"5_zone"`,
- `"10_zone"`,
- `"10_zone_compound"`,
- `"10_zone_6_ring"`,
- `"10_zone_5_ring"`,
- `"10_zone_5_ring_compound"`,
- `"WA_field"`,
- `"IFAA_field"`,
- `"IFAA_field_expert"`,
- `"Beiter_hit_miss"`,
- `"Worcester"`,
- `"Worcester_2_ring"`

where an `"_n_ring"` suffix indicates a reduced scoring area with only the central n rings, and the `"_compound"` suffix indicates the system where only the x-ring scores 10 points.

So we could define the target shot on the WA 720 70m round as:

In [ ]:
my720target = au.targets.Target(1.22, "10_zone", 70.0)

and the corresponding target shot by compounds (reduced 80cm face at 50m) as:

In [ ]:
mycompound720target = au.targets.Target(0.80, "10_zone_6_ring", 50.0)

The target object can also take the optional arguments of:
- `native_dist_unit` - string - default = `"metres"`
- `indoor` - boolean - default = `False`

to provide an imperial distance in yards and indicate if the round is to be shot indoors (where rules may be different).

For example, the longest target on an IFAA field rounds could be defined as follows:

In [ ]:
myIFAATarget = au.targets.Target(0.80, "IFAA_field", 80.0, native_dist_unit="yards")

and the target shot for the Archery GB Postsmouth round as:

In [ ]:
myPortsmouthTarget = au.targets.Target(
    0.60, "10_zone", 20.0, native_dist_unit="yards", indoor=True
)

Target objects have the ability to return the maximum possible score from the type of face specified through the `max_score` method:

In [ ]:
for target in [my720target, mycompound720target, myIFAATarget, myPortsmouthTarget]:
    print(target.max_score())

### Pass
The natural extension to the `targets.Target` class is to shoot a number of arrows at it.
in _archeryutils_ this is called a "pass" and is defined using the `rounds.Pass` class which wraps around the `targets.Target` class.

This takes a number of arrows followed by all of the arguments to target defined above.

For example, to define the 36 arrow pass that forms the first distance on a WA 1440 70m round, or the first half of the WA 720 70m round we use:

In [ ]:
my70mPass = au.rounds.Pass(36, 1.22, "10_zone", 70.0)

Like the `targets.Target` class a `rounds.Pass` also has a `max_score()` method, but this now returns the maximum possible score for the pass (`n_arrows * Target.max_score()`):

In [ ]:
print(my70mPass.max_score())

### Round
In reality we rarely use the `targets.Target` or `rounds.Pass` objects by themselves, however, instead preferring to use the `rounds.Round` class. This defines multiple passes to form what is commonly known as a round.

A `rounds.Round` object is defined with a string `name` to provide a popular name for the round and a list of `rounds.Pass` objects.

It may also take the following optional string arguments:
- `location` - where the round is shot, e.g. 'Indoor', 'Outdoor', 'Field' etc.
- `body` - The governing body the round is defined by, e.g. 'WA', 'IFAA', 'AGB', 'AA' etc.
- `family` - The larger family of rounds to which this round belongs, e.g. 'wa_1440', 'wa_720', 'national' etc.

So to define a WA 720 70m round we can re-use our variable `my70mPass` from above as follows:

In [ ]:
my720Round = au.rounds.Round(
    "WA 720 (70m)",
    [my70mPass, my70mPass],
    location="Outdoor Target",
    body="WA",
    family="WA720",
)

Again we have a method for maximum score:

In [ ]:
print(my720Round.max_score())

### Default Rounds

A number of useful rounds are pre-defined and come preloaded as dictionaries that can be imported:

In [ ]:
from archeryutils import load_rounds

agb_outdoor = load_rounds.AGB_outdoor_imperial

for round_i in agb_outdoor.values():
    print(round_i.name)

The individial rounds are accessible via 'dot' notation (using the alias listed in `agb_outdoor.keys()`) as follows:

In [ ]:
agb_outdoor.york.get_info()

agb_outdoor.york.max_score()

Possible options for round collections are:
- `AGB_outdoor_imperial` - Archery GB outdoor imperial rounds
- `AGB_outdoor_metric` - Archery GB outdoor metric rounds
- `AGB_indoor` - Archery GB indoor rounds
- `WA_outdoor` - World Archery outdoor rounds
- `WA_indoor` - World Archery indoor rounds
- `WA_field` - World Archery field rounds
- `IFAA_field` - IFAA indoor and outdoor rounds
- `AGB_VI` - Archery GB Visually Impaired rounds
- `WA_VI` - World Archery Visually Impaired rounds
- `custom` - custom rounds such as individual distances, 252 awards, frostbites etc.

## 2. Handicap Schemes
_archeryutils_ provides functionality for calculating various [handicaps/skill ratings](https://jackatkinson.net/post/archery_handicap/) from scores. These include both the popular Archery GB and Archery Australia schemes.

To use these functionalities import the handicap equations and functions modules as below.

It will also help to instantiate a `HcParams` object that contains key parameters for the handicap schemes.
This comes as a preloaded dataclass, though the values of the variables can be changed or loaded from a `.json` file. For an explanation of the different values consult the [class definition in `archeryutils/handicaps/handicap_equations.py`](https://github.com/jatkinson1000/archeryutils/blob/main/archeryutils/handicaps/handicap_equations.py).

In [ ]:
from archeryutils import handicap_equations as hc_eq
from archeryutils import handicap_functions as hc_func

hcparams = hc_eq.HcParams()
print(hcparams)

### Score from handicap

It is then possible to use the `score_for_round` function to calculate score on any `rounds.Round` for a given handicap/skill rating.

This requires a round, handicap/skill rating, scheme, and set of handicap parameters.

Possible options for the scheme are:
- `"AGB"` - The 2023 Archery GB handicap system developed by Jack Atkinson
- `"AGBold"` - The old Archery GB handicap system developed by David Lane
- `"AA2"` - The 2014 Archery Australia Skill rating system developed by Jim Park
- `"AA"` - The old Archery Australia skill rating system developed by Jim Park

For example, to calculate the score on a York round for a handicap of 38 using the 2023 Archery GB scheme we run:

In [ ]:
score_from_hc, _ = hc_eq.score_for_round(
    agb_outdoor.york,
    38,
    "AGB",
    hcparams,
)

print(f"A handicap of 38 on a York is a score of {score_from_hc}.")

Note that it is possible to obtain scores for decimal handicaps:

In [ ]:
score_from_hc, _ = hc_eq.score_for_round(
    agb_outdoor.york,
    38.25,
    "AGB",
    hcparams,
)

print(f"A handicap of 38.25 on a York is a score of {score_from_hc}.")

By default this function returns a round score as would appear in handicap tables and is physically attainable when shooting a round. The rounding mechanism (round/floor/ceil) varies by scheme. However, it is possible to return the mathematically continuous score by setting the `round_score_up` optional argument to be `False`:

In [ ]:
score_from_hc, _ = hc_eq.score_for_round(
    agb_outdoor.york,
    38.25,
    "AGB",
    hcparams,
    round_score_up=False,
)

print(f"A handicap of 38.25 on a York is a decimal score of {score_from_hc}.")

### Handicap from Score

Mathematically is is easy to define a score for a given handicap, but often the opposite is required, where one wishes to obtain the handicap given a score.

To perform this operation use the `handicap_from_score()` function which takes a score, round, handicap scheme, and set of parameters.
By default it returns the decimal handicap corresponding to the provided score exactly.
However, it is possible to return the integer handicap value that the score would correspond to in a handicap table by setting the `int_prec` optional argument to `True`. Remember that the rounding mechanism (round/floor/ceil) varies by scheme.


For example, to get the 2023 Archery GB handicap given by a score of 950 on a York round:

In [ ]:
hc_from_score = hc_func.handicap_from_score(
    950,
    agb_outdoor.york,
    "AGB",
    hcparams,
)
print(f"A score of 950 on a York is a continuous handicap of {hc_from_score}.")

hc_from_score = hc_func.handicap_from_score(
    950,
    agb_outdoor.york,
    "AGB",
    hcparams,
    int_prec=True,
)
print(f"A score of 950 on a York is a discrete handicap of {hc_from_score}.")

### Handicap Tables

A further functionality of the 

In [ ]:
import numpy as np

handicaps = np.arange(0.0, 151.0, 1.0)
rounds = [
    agb_outdoor.york,
    agb_outdoor.hereford,
    agb_outdoor.albion,
    agb_outdoor.windsor,
]
# The following allows printing of handicap tables for an entire group of rounds:
# rounds = list(load_rounds.AGB_outdoor_imperial.values())

hc_func.print_handicap_table(
    handicaps,
    "AGB",
    rounds,
    hcparams,
)

The following optional arguments can be passed to `print_handicap_table()`:
- `round_scores_up` - round scores to discrete values as appropriate for the scheme
- `clean_gaps` - if `True` duplicate scores will be displayed for the first occurrence only
- `filename` - if provided as a string the table will be saved to file
- `int_prec` - if `True` then values will be printed as integers rather than decimals

It is also possible to pass an array of non-integer handicaps.

The effect of these variables can be examined by changing their values in the following:

In [ ]:
handicaps = np.arange(0.0, 51.0, 0.5)


hc_func.print_handicap_table(
    handicaps,
    "AGB",
    rounds,
    hcparams,
    round_scores_up=False,
    clean_gaps=False,
    filename="test_handicap_table.txt",
    int_prec=False,
)

## 3. Classifications

As well as handicap functionalities _archeryutils_ fontains functionalities for calculating Archery GB classifications.
These are accessed by importing the `classifications` module:

In [ ]:
from archeryutils import classifications as class_func

### Classification from score

To get a classification that results from a score use the `calculate_X_classification()` function, where `X` corresponds to the classification scheme being used (`AGB_outdoor`, `AGB_indoor`, `AGB_field`).

This takes following arguments:
- a string of a round alias (see [Default Rounds](#Default-Rounds)).
- a score
- a string of bowstyle (`"compound"`, `"recurve"`, `"longbow"`, `"barebow"`, `"traditional"`, `"flatbow"`)
- a string of gender under Archery GB (`"male"` or `"female"`)
- an Archery GB age group (`"50+"`, `"adult"`, `"under 21"`, `"under 18"`, etc.)

and returns a string corresponding to the classification it obtains.

These can be investigated in the following code snippet which uses a number of examples:

In [ ]:
# AGB Outdoor
class_from_score = class_func.calculate_agb_outdoor_classification(
    "hereford",
    965,
    "recurve",
    "male",
    "50+",
)
print(
    f"A score of 965 on a Hereford is class {class_from_score} for a 50+ male recurve."
)

In [ ]:
# AGB Indoor
class_from_score = class_func.calculate_agb_indoor_classification(
    "wa18",
    562,
    "compound",
    "female",
    "adult",
)
print(
    f"A score of 562 on a WA 18 is class {class_from_score} for adult female compound."
)

In [ ]:
# AGB Field
class_from_score = class_func.calculate_agb_field_classification(
    "wa_field_24_blue_unmarked",
    168,
    "traditional",
    "male",
    "under 18",
)
print(
    f"A score of 168 on a WA Unmarked 24 is class {class_from_score} for an under 18 male traditional."
)

### Classification scores

As well as generating a classification from a score there is also the inverse functionality of obtaining scores required for classifications. This can be done using the `X_classification_scores()` functions.

These take a round alias and categories as strings above, and return a list of scores required for each classification in descending order.

Where a classification is not available from a particular round a fill value of -9999 is returned.

In [ ]:
class_scores = class_func.agb_outdoor_classification_scores(
    "hereford",
    "recurve",
    "male",
    "adult",
)
print(class_scores)

In [ ]:
class_scores = class_func.agb_indoor_classification_scores(
    "portsmouth",
    "compound",
    "female",
    "adult",
)
print(class_scores)

In [ ]:
class_scores = class_func.agb_field_classification_scores(
    "wa_field_24_blue_marked",
    "flatbow",
    "female",
    "under 18",
)
print(class_scores)